In [1]:
RUN_TEST = True # Do a fast run to check the pipeline is working
RUN_1_BATCH = False # Do a fast run to check the pipeline is working
MAX_EPOCH = 10

RUN_NAME = 'TEST_1' # WandB run name
PROJECT = 'Last_V1' # WandB project name, monitor audio outputs in real time
HARDWARE = 'CPU' # T4 or P100 or CPU
DATA_SAVE = 'data' # Directory to save data
DATA_PATH = r'C:\Users\adhem\Desktop\Music\Custom_VST\data\ARME-Virtuoso-Strings-2.2' # Directory to the dataset
KAGGLE = False

In [2]:
import sys
import os
sys.path.append(os.path.abspath('c:/Users/adhem/Desktop/Music/Custom_VST/code'))

In [3]:
from launch import TrainerWarper
import torch.nn as nn

#### Model Config

In [4]:
model_cfg = {
    'in_channels': 1,
    'out_channels': 16,
    'channels': [16, 32, 64, 64],
    'factors': [4, 4, 4, 4],
    'res_blocks': 2,
    'activation': nn.SiLU,
    'use_norm': True,
    'num_groups': 4,
    'variational': True,
    'lstm': True,
}

#### Optimizer Config

In [5]:
optimizer_cfg = {
    'lr': 0.001,
    'betas': (0.9, 0.999),
    'weight_decay': 0,
}

#### Loss Config

In [6]:
loss_cfg = {
    'L1TemporalLoss':{
        'weight':1,
        'balancer': True,
        'key_output': 'x_hat',
        'key_target': 'x',
        },
    'L2TemporalLoss':{
        'weight':1,
        'balancer': True,
        'key_output': 'x_hat',
        'key_target': 'x',
        },
    'AuralossLoss':{
        'weight':1,
        'balancer': True,
        'key_output': 'x_hat',
        'key_target': 'x',
        "fft_sizes": [32, 128, 512, 2048],  # [32, 128, 512, 2048, 8192, 32768]
        "hop_sizes": [16, 64, 256, 1024],  # [16, 64, 256, 1024, 4096, 16384]
        "win_lengths": [32, 128, 512, 2048],  # [32, 128, 512, 2048, 8192, 32768]
        "w_sc": 0.0,
        "w_phs": 0.0,
        "w_lin_mag": 1.0,
        "w_log_mag": 1.0,
    },
    'KLDivergenceLoss':{
        'weight':1,
        'balancer': False,
        'key_mean': 'z_mean',
        'key_log_std': 'z_log_std',
        },
    'Discriminator':{
        'weight':1,
        'balancer': True,
        #'type':'Spectrogram_VGGStyle',
    },
    'FeatureLoss':{
        'weight':1,
        'balancer': True,
    },
}

#### Data Config

In [7]:
data_cfg = {
    'path': DATA_PATH,
    'sr': 22050,
    'sample_duration':8, # Seconds
    'latent_compression': 4**4,
    'envelope_detector':{
        'type': 'Max',
        },
    'pitch_detector': {
        'n_fft': 4096,
        'hop_length': 1024,
        'fmin': 75,
        'fmax': 800,
        'threshold': 0.15,
        'win_length': None,  # Defaults to n_fft
        'window': 'hann',
        'center': True,
        'pad_mode': 'reflect'
    },
    'batch_size': 8,
    'num_workers': 2,
}

#### Training Config

In [8]:
if KAGGLE:
    # Kaggle herdware
    if HARDWARE == 'T4':
        num_gpus = 2
        available_devices = [0,1]
        print('On Kaggle Double T4')
    elif HARDWARE == 'P100':
        num_gpus = 1
        available_devices = [0]
        print('On Kaggle P100')
    else:
        num_gpus = 0
        available_devices = ['CPU']
        print('On Kaggle CPU')
else:
    num_gpus = 0 # No gpu on my computer :(
    available_devices = ['CPU']
    print('On laptop CPU')

accelerator = 'gpu' if num_gpus > 0 else 'cpu'
devices = num_gpus if num_gpus > 0 else 1

On laptop CPU


In [9]:
# Training configuration
training_cfg = {
    
    # Hardware
    'accelerator': accelerator,
    'devices': devices,
    'num_nodes': 1,
    'precision': 32,  # Float32
    'deterministic': False,  # Increases training speed for fixed tensor sizes
    'benchmark': True,  # Optimizes for fixed tensor size dataset
    
    # Epochs
    'min_epochs': 4,
    'max_epochs': MAX_EPOCH,
    'max_time': '00:12:00:00',
    'accumulate_grad_batches': 1,
    
    # Callbacks
    'callbacks': [
    ('EarlyStopping', 
     {
        'monitor': 'train_loss',
        'patience': 5, 
        'verbose': True, 
        'mode': 'min'
    }),

    ('ModelCheckpoint', 
     {
        'monitor': 'train_loss', 
        'dirpath': f'{DATA_SAVE}/', 
        'filename': 'best-checkpoint', 
        'save_top_k': 1, 
        'mode': 'min'
    }),
    ],
    
    # Logging / Debug
    'logger': None,  # To be defined later
    'profiler': 'simple',
    'fast_dev_run': RUN_TEST,
    'limit_train_batches': 1 if RUN_1_BATCH else None,
    'enable_checkpointing': True,
    'barebones': False,
}

#### Wandb

In [10]:
import wandb
from pytorch_lightning.loggers import WandbLogger

all_config = {
    'TRAINING': training_cfg,
    'MODEL': model_cfg,
    'DATA': data_cfg,
    'TRAINER': training_cfg,
}

training_cfg['logger'] = None if (RUN_TEST or RUN_1_BATCH) else WandbLogger(project=PROJECT, name=RUN_NAME, config=all_config)

In [11]:
TRAINER = TrainerWarper(
    model_cfg,
    data_cfg,
    optimizer_cfg,
    loss_cfg,
    training_cfg
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\adhem\anaconda3\envs\Audio\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [12]:
TRAINER.train()


  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | model | Autoencoder1d | 506 K  | train
------------------------------------------------
506 K     Trainable params
0         Non-trainable params
506 K     Total params
2.026     Total estimated model params size (MB)
189       Modules in train mode
0         Modules in eval mode
c:\Users\adhem\anaconda3\envs\Audio\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] 0 torch.Size([8, 32, 513, 690])
1 torch.Size([8, 32, 513, 345])
2 torch.Size([8, 32, 511, 173])
3 torch.Size([8, 32, 505, 87])
4 torch.Size([8, 32, 503, 85])
torch.Size([8, 501, 83])
torch.Size([8, 81])
0 torch.Size([8, 32, 513, 690])
1 torch.Size([8, 32, 513, 345])
2 torch.Size([8, 32, 511, 173])
3 torch.Size([8, 32, 505, 87])
4 torch.Size([8, 32, 503, 85])
torch.Size([8, 501, 83])
torch.Size([8, 81])
